In [2]:
import numpy as np
import pandas as pd

# -------------------------------
# Explicit projection onto probability simplex
# -------------------------------
def project_simplex(w):
    """Projects a vector onto the probability simplex."""
    w = np.maximum(w, 1e-5)
    w /= np.sum(w)
    return w

# -------------------------------
# Hilbert-enhanced CVaR update (vector-based)
# -------------------------------
def hilbert_cvar_update(weights, asset_returns, alpha=0.05):
    """
    CVaR-adjusted update with asset-level vector risk consideration and explicit projection.
    """
    losses = -asset_returns
    var = np.percentile(losses, 100 * (1 - alpha))
    cvar = losses[losses >= var].mean()
    adj = np.exp(-cvar)

    new_w = weights * adj
    new_w = project_simplex(new_w)

    return new_w

# -------------------------------
# Tests
# -------------------------------
def test_projection(weights):
    assert np.isclose(np.sum(weights), 1.0, atol=1e-6), "Weights do not sum to 1"
    assert np.all(weights >= 0), "Negative weights found"

def test_convergence(weights_history):
    diffs = np.abs(np.diff(weights_history, axis=0))
    mean_change = np.mean(diffs)
    print(f"✅ Mean weight change per update: {mean_change:.6f}")

def test_cvar_reduction(returns, weights_initial, weights_final, alpha=0.05):
    port_ret_initial = returns @ weights_initial
    port_ret_final = returns @ weights_final

    losses_initial = -port_ret_initial
    losses_final = -port_ret_final

    var_init = np.percentile(losses_initial, 100 * (1 - alpha))
    var_final = np.percentile(losses_final, 100 * (1 - alpha))

    cvar_init = losses_initial[losses_initial >= var_init].mean()
    cvar_final = losses_final[losses_final >= var_final].mean()

    print(f"✅ Initial CVaR: {cvar_init:.4f}, Final CVaR: {cvar_final:.4f}")

def test_smoothness(weights, returns, eps=1e-5):
    base_value = hilbert_cvar_update(weights, returns)
    perturbed_weights = weights + eps
    perturbed_weights = project_simplex(perturbed_weights)
    pert_value = hilbert_cvar_update(perturbed_weights, returns)
    grad_norm = np.linalg.norm(pert_value - base_value) / eps
    print(f"✅ Approx gradient norm: {grad_norm:.6f}")

# -------------------------------
# Simulation setup
# -------------------------------
np.random.seed(42)
dates = pd.date_range("2010-01-01", periods=2520, freq='B')
assets = ["AAPL", "MSFT", "GOOG", "AMZN", "META"]
returns_df = pd.DataFrame(0.001 + 0.02 * np.random.randn(len(dates), len(assets)), index=dates, columns=assets)

weights = np.ones(len(assets)) / len(assets)
weights_history = []

for t in range(60, len(returns_df)):
    asset_returns = returns_df.iloc[t].values  # Now using full vector
    adj_weights = hilbert_cvar_update(weights, asset_returns)
    weights = adj_weights
    weights_history.append(weights)

weights_history = np.array(weights_history)

# -------------------------------
# Run tests
# -------------------------------
test_projection(weights_history[-1])
test_convergence(weights_history)
test_cvar_reduction(returns_df.iloc[-100:].values, np.ones(len(assets)) / len(assets), weights_history[-1])
test_smoothness(weights, returns_df.iloc[-1].values)

print("✅ All improved tests completed successfully!")


✅ Mean weight change per update: 0.000000
✅ Initial CVaR: 0.0139, Final CVaR: 0.0139
✅ Approx gradient norm: 0.000000
✅ All improved tests completed successfully!


In [3]:
import numpy as np
import pandas as pd

# -------------------------------
# Explicit projection operator
# -------------------------------
def project_simplex(w):
    w = np.maximum(w, 1e-5)
    w /= np.sum(w)
    return w

# -------------------------------
# Hilbert-enhanced CVaR update
# -------------------------------
def hilbert_cvar_update(weights, asset_returns, alpha=0.05):
    losses = -asset_returns
    var = np.percentile(losses, 100 * (1 - alpha))
    cvar = losses[losses >= var].mean()
    adj = np.exp(-cvar)
    new_w = weights * adj
    new_w = project_simplex(new_w)
    return new_w

# -------------------------------
# Tests
# -------------------------------
def test_projection(weights):
    assert np.isclose(np.sum(weights), 1.0, atol=1e-6), "Weights do not sum to 1"
    assert np.all(weights >= 0), "Negative weights found"

def test_convergence(weights_history):
    diffs = np.abs(np.diff(weights_history, axis=0))
    mean_change = np.mean(diffs)
    print(f"✅ Mean weight change per update: {mean_change:.6f}")

def test_cvar_reduction(returns, weights_initial, weights_final, alpha=0.05):
    port_ret_initial = returns @ weights_initial
    port_ret_final = returns @ weights_final
    losses_initial = -port_ret_initial
    losses_final = -port_ret_final
    var_init = np.percentile(losses_initial, 100 * (1 - alpha))
    var_final = np.percentile(losses_final, 100 * (1 - alpha))
    cvar_init = losses_initial[losses_initial >= var_init].mean()
    cvar_final = losses_final[losses_final >= var_final].mean()
    print(f"✅ Initial CVaR: {cvar_init:.4f}, Final CVaR: {cvar_final:.4f}")

def test_smoothness(weights, returns, eps=1e-5):
    base_value = hilbert_cvar_update(weights, returns)
    perturbed_weights = weights + eps
    perturbed_weights = project_simplex(perturbed_weights)
    pert_value = hilbert_cvar_update(perturbed_weights, returns)
    grad_norm = np.linalg.norm(pert_value - base_value) / eps
    print(f"✅ Approx gradient norm: {grad_norm:.6f}")

# -------------------------------
# Heavy-tail & shock simulation
# -------------------------------
np.random.seed(42)
dates = pd.date_range("2010-01-01", periods=2520, freq='B')
assets = ["AAPL", "MSFT", "GOOG", "AMZN", "META"]

# Simulate t-distributed returns (df=3 for heavy tails)
returns_arr = 0.001 + 0.03 * np.random.standard_t(df=3, size=(len(dates), len(assets)))

# Crisis shocks: inject big negative returns on random days
shock_days = np.random.choice(len(dates), size=10, replace=False)
for day in shock_days:
    returns_arr[day] -= 0.2  # Large negative shock

returns_df = pd.DataFrame(returns_arr, index=dates, columns=assets)

# -------------------------------
# Run optimization
# -------------------------------
weights = np.ones(len(assets)) / len(assets)
weights_history = []

for t in range(60, len(returns_df)):
    asset_returns = returns_df.iloc[t].values
    adj_weights = hilbert_cvar_update(weights, asset_returns)
    weights = adj_weights
    weights_history.append(weights)

weights_history = np.array(weights_history)

# -------------------------------
# Run tests
# -------------------------------
test_projection(weights_history[-1])
test_convergence(weights_history)
test_cvar_reduction(returns_df.iloc[-100:].values, np.ones(len(assets)) / len(assets), weights_history[-1])
test_smoothness(weights, returns_df.iloc[-1].values)

print("✅ All heavy-tail stress tests completed successfully!")


✅ Mean weight change per update: 0.000000
✅ Initial CVaR: 0.0499, Final CVaR: 0.0499
✅ Approx gradient norm: 0.000000
✅ All heavy-tail stress tests completed successfully!


In [4]:
import numpy as np
import pandas as pd

# -------------------------------
# Projection operator
# -------------------------------
def project_simplex(w):
    w = np.maximum(w, 1e-5)
    w /= np.sum(w)
    return w

# -------------------------------
# Portfolio CVaR
# -------------------------------
def portfolio_cvar(weights, returns_matrix, alpha=0.05):
    portfolio_returns = returns_matrix @ weights
    losses = -portfolio_returns
    var = np.percentile(losses, 100 * (1 - alpha))
    cvar = losses[losses >= var].mean()
    return cvar

# -------------------------------
# Marginal CVaR computation
# -------------------------------
def compute_marginal_cvar(weights, returns_matrix, alpha=0.05, epsilon=1e-4):
    base_cvar = portfolio_cvar(weights, returns_matrix, alpha)
    marginal_cvar = np.zeros_like(weights)

    for i in range(len(weights)):
        perturbed = weights.copy()
        perturbed[i] += epsilon
        perturbed = project_simplex(perturbed)
        pert_cvar = portfolio_cvar(perturbed, returns_matrix, alpha)
        marginal_cvar[i] = (pert_cvar - base_cvar) / epsilon

    return marginal_cvar

# -------------------------------
# Hilbert-enhanced marginal CVaR update
# -------------------------------
def hilbert_marginal_cvar_update(weights, returns_matrix, alpha=0.05):
    marginal_cvar = compute_marginal_cvar(weights, returns_matrix, alpha)
    adj = np.exp(-marginal_cvar)
    new_w = weights * adj
    new_w = project_simplex(new_w)
    return new_w

# -------------------------------
# Tests
# -------------------------------
def test_projection(weights):
    assert np.isclose(np.sum(weights), 1.0, atol=1e-6), "Weights do not sum to 1"
    assert np.all(weights >= 0), "Negative weights found"

def test_convergence(weights_history):
    diffs = np.abs(np.diff(weights_history, axis=0))
    mean_change = np.mean(diffs)
    print(f"✅ Mean weight change per update: {mean_change:.6f}")

def test_cvar_reduction(returns_matrix, weights_initial, weights_final, alpha=0.05):
    cvar_init = portfolio_cvar(weights_initial, returns_matrix, alpha)
    cvar_final = portfolio_cvar(weights_final, returns_matrix, alpha)
    print(f"✅ Initial CVaR: {cvar_init:.4f}, Final CVaR: {cvar_final:.4f}")

def test_smoothness(weights, returns_matrix, eps=1e-5):
    base_value = hilbert_marginal_cvar_update(weights, returns_matrix)
    perturbed_weights = weights + eps
    perturbed_weights = project_simplex(perturbed_weights)
    pert_value = hilbert_marginal_cvar_update(perturbed_weights, returns_matrix)
    grad_norm = np.linalg.norm(pert_value - base_value) / eps
    print(f"✅ Approx gradient norm: {grad_norm:.6f}")

# -------------------------------
# Heavy-tail & shock simulation
# -------------------------------
np.random.seed(42)
dates = pd.date_range("2010-01-01", periods=2520, freq='B')
assets = ["AAPL", "MSFT", "GOOG", "AMZN", "META"]

# Simulate heavy-tailed returns (t-distributed, df=3)
returns_arr = 0.001 + 0.03 * np.random.standard_t(df=3, size=(len(dates), len(assets)))

# Inject large negative shocks
shock_days = np.random.choice(len(dates), size=10, replace=False)
for day in shock_days:
    returns_arr[day] -= 0.2

returns_df = pd.DataFrame(returns_arr, index=dates, columns=assets)

# -------------------------------
# Run optimization loop
# -------------------------------
weights = np.ones(len(assets)) / len(assets)
weights_history = []

# Use rolling window for marginal CVaR estimates
window_size = 60

for t in range(window_size, len(returns_df)):
    returns_window = returns_df.iloc[t - window_size:t].values
    adj_weights = hilbert_marginal_cvar_update(weights, returns_window)
    weights = adj_weights
    weights_history.append(weights)

weights_history = np.array(weights_history)

# -------------------------------
# Run tests
# -------------------------------
test_projection(weights_history[-1])
test_convergence(weights_history)
test_cvar_reduction(returns_df.iloc[-100:].values, np.ones(len(assets)) / len(assets), weights_history[-1])
test_smoothness(weights, returns_df.iloc[-60:].values)

print("✅ All marginal CVaR tests completed successfully!")


✅ Mean weight change per update: 0.003912
✅ Initial CVaR: 0.0499, Final CVaR: 0.0383
✅ Approx gradient norm: 1.103785
✅ All marginal CVaR tests completed successfully!


In [5]:
import numpy as np
import pandas as pd

# -------------------------------
# Projection operator
# -------------------------------
def project_simplex(w):
    w = np.maximum(w, 1e-5)
    w /= np.sum(w)
    return w

# -------------------------------
# Portfolio CVaR
# -------------------------------
def portfolio_cvar(weights, returns_matrix, alpha=0.05):
    portfolio_returns = returns_matrix @ weights
    losses = -portfolio_returns
    var = np.percentile(losses, 100 * (1 - alpha))
    cvar = losses[losses >= var].mean()
    return cvar

# -------------------------------
# Marginal CVaR gradient
# -------------------------------
def compute_marginal_cvar(weights, returns_matrix, alpha=0.05, epsilon=1e-4):
    base_cvar = portfolio_cvar(weights, returns_matrix, alpha)
    marginal_cvar = np.zeros_like(weights)

    for i in range(len(weights)):
        perturbed = weights.copy()
        perturbed[i] += epsilon
        perturbed = project_simplex(perturbed)
        pert_cvar = portfolio_cvar(perturbed, returns_matrix, alpha)
        marginal_cvar[i] = (pert_cvar - base_cvar) / epsilon

    return marginal_cvar

# -------------------------------
# Continuous-time-inspired update
# -------------------------------
def continuous_cvar_update(weights, returns_matrix, alpha=0.05, eta=0.01):
    grad = compute_marginal_cvar(weights, returns_matrix, alpha)
    new_w = weights - eta * grad
    new_w = project_simplex(new_w)
    return new_w

# -------------------------------
# Heavy-tail & shock simulation
# -------------------------------
np.random.seed(42)
dates = pd.date_range("2010-01-01", periods=2520, freq='B')
assets = ["AAPL", "MSFT", "GOOG", "AMZN", "META"]

returns_arr = 0.001 + 0.03 * np.random.standard_t(df=3, size=(len(dates), len(assets)))
shock_days = np.random.choice(len(dates), size=10, replace=False)
for day in shock_days:
    returns_arr[day] -= 0.2

returns_df = pd.DataFrame(returns_arr, index=dates, columns=assets)

# -------------------------------
# Run continuous-time-inspired optimization
# -------------------------------
weights = np.ones(len(assets)) / len(assets)
weights_history = []

window_size = 60

for t in range(window_size, len(returns_df)):
    returns_window = returns_df.iloc[t - window_size:t].values
    weights = continuous_cvar_update(weights, returns_window, eta=0.01)
    weights_history.append(weights)

weights_history = np.array(weights_history)

# -------------------------------
# Tests
# -------------------------------
def test_projection(weights):
    assert np.isclose(np.sum(weights), 1.0, atol=1e-6), "Weights do not sum to 1"
    assert np.all(weights >= 0), "Negative weights found"

def test_convergence(weights_history):
    diffs = np.abs(np.diff(weights_history, axis=0))
    mean_change = np.mean(diffs)
    print(f"✅ Mean weight change per update: {mean_change:.6f}")

def test_cvar_reduction(returns_matrix, weights_initial, weights_final, alpha=0.05):
    cvar_init = portfolio_cvar(weights_initial, returns_matrix, alpha)
    cvar_final = portfolio_cvar(weights_final, returns_matrix, alpha)
    print(f"✅ Initial CVaR: {cvar_init:.4f}, Final CVaR: {cvar_final:.4f}")

def test_smoothness(weights, returns_matrix, eps=1e-5):
    base_value = continuous_cvar_update(weights, returns_matrix, eta=0.01)
    perturbed_weights = weights + eps
    perturbed_weights = project_simplex(perturbed_weights)
    pert_value = continuous_cvar_update(perturbed_weights, returns_matrix, eta=0.01)
    grad_norm = np.linalg.norm(pert_value - base_value) / eps
    print(f"✅ Approx gradient norm: {grad_norm:.6f}")

# Run final tests
test_projection(weights_history[-1])
test_convergence(weights_history)
test_cvar_reduction(returns_df.iloc[-100:].values, np.ones(len(assets)) / len(assets), weights_history[-1])
test_smoothness(weights, returns_df.iloc[-60:].values)

print("✅ All continuous-time-inspired tests completed successfully!")


✅ Mean weight change per update: 0.000326
✅ Initial CVaR: 0.0499, Final CVaR: 0.0419
✅ Approx gradient norm: 0.539012
✅ All continuous-time-inspired tests completed successfully!
